## Item 19: Never Unpack More Than Three Variables When Functions Return Multiple Values

✦ Unpacking into four or more variables is error prone and should be avoided; instead, return a small class or namedtuple instance.

## Item 20: Prefer Raising Exceptions to Returning None

✦ Functions that return None to indicate special meaning are error prone because None and other values (e.g., zero, the empty string) all evaluate to False in conditional expressions.

## Item 21: Know How Closures Interact with Variable Scope

This function generates an unexpected result:
    
```python
def sort_priority2(numbers, group):
    found = False        # Scope: 'sort_priority2'
    def helper(x):
        if x in group:
            found = True # Scope: 'helper' -- Bad!
            return (0, x)
        return (1, x)

    numbers.sort(key=helper)
    return found

found = sort_priority2(numbers, group)
print('Found:', found)
print(numbers)

>>>
Found: False
[2, 3, 5, 7, 1, 4, 6, 8]
```

To avoid the issue, you can use `local`:

```python
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found # Added
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found
```

Or, you can create a helper class:

```python
class Sorter:
    def __init__(self, group):

        self.group = group
        self.found = False

    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
    return (1, x)

sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True
```

When you reference a variable in an expression, the Python interpreter traverses the scope to resolve the reference in this order:

  1. The current function’s scope.
  2. Any enclosing scopes (such as other containing functions).
  3. The scope of the module that contains the code (also called the global scope).
  4. The built-in scope (that contains functions like len and str).

## Item 22: Reduce Visual Noise with Variable Positional Arguments

The positional arguments are often called _varargs_ for short, or _star args_, in reference to the conventional name for the parameter `*args`.

In [3]:
def log(message, *values): 
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log('My numbers are', 1, 2)
log('Hi there')

My numbers are: 1, 2
Hi there


In [5]:
# Using the * operator with a generator may cause 
#  a program to run out of memory and crash.
def my_generator():
    for i in range(10):
        yield i

def my_func(*args):
   print(args)

it = my_generator()
my_func(*it)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


## Item 23: Provide Optional Behavior with Keyword Arguments

In [6]:
def remainder(number, divisor):
    return number % divisor

print(remainder(20, 7))
print(remainder(20, divisor=7))
print(remainder(number=20, divisor=7))
print(remainder(divisor=7, number=20))

6
6
6
6


In [8]:
# kwargs = keyword arguments
my_kwargs = {
    'number': 20,
    'divisor': 7,
}

print(remainder(**my_kwargs))

6


In [9]:
my_kwargs = {
    'divisor': 7,
}

print(remainder(number=20, **my_kwargs))

6


In [10]:
my_kwargs = {
    'number': 20,
}
other_kwargs = {
    'divisor': 7,
}

print(remainder(**my_kwargs, **other_kwargs))

6


In [11]:
def print_parameters(**kwargs):
    for key, value in kwargs.items():
        print(f'{key} = {value}')

print_parameters(alpha=1.5, beta=9, gamma=4)

alpha = 1.5
beta = 9
gamma = 4


## Item 24: Use None and Docstrings to Specify Dynamic Default Arguments

In [12]:
from time import sleep
from datetime import datetime

# This doesn't work as expected as datetime.now is excuted only once.
def log(message, when=datetime.now()):
    print(f'{when}: {message}')

log('Hi there!')
sleep(0.1)
log('Hello again!')

2019-12-31 16:20:38.928119: Hi there!
2019-12-31 16:20:38.928119: Hello again!


In [13]:
# Fix
def log(message, when=None):
    """Log a message with a timestamp.

    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')
    
log('Hi there!')
sleep(0.1)
log('Hello again!')

2019-12-31 16:21:36.400700: Hi there!
2019-12-31 16:21:36.502143: Hello again!


In [18]:
# Fix with type annotations
from typing import Optional

def log_typed(message: str,
              when: Optional[datetime]=None) -> None:
    """Log a message with a timestamp.

    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')
    
log('Hi there!')
sleep(0.1)
log('Hello again!')

2019-12-31 16:26:23.900533: Hi there!
2019-12-31 16:26:24.002201: Hello again!


In [15]:
import json

# Using None for default argument values is 
#  especially important when the arguments are mutable. 
def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default
    
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)
print(foo == bar)

Foo: {'stuff': 5, 'meep': 1}
Bar: {'stuff': 5, 'meep': 1}
True


In [19]:
# Fix
def decode(data, default=None):
    """Load JSON data from a string.

    Args:
         data: JSON data to decode.
         default: Value to return if decoding fails.
             Defaults to an empty dictionary.
    """
    try:
         return json.loads(data)
    except ValueError:
         if default is None:
             default = {}
    return default

foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)
print(foo == bar)

Foo: {'stuff': 5}
Bar: {'meep': 1}
False


## Item 25: Enforce Clarity with Keyword-Only and Positional-Only Arguments

In [32]:
# The `*` symbol in the argument list indicates 
#  the end of positional arguments and 
#  the beginning of keyword-only arguments:
def safe_division(number, divisor, *,
                  ignore_overflow=False,
                  ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
             return 0
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

            
try:
    safe_division(1.0, 0)
except Exception as e:
    print(repr(e))

ZeroDivisionError('float division by zero')


Python 3.8 introduces positional-only arguments.
The / symbol in the argument list indicates 
 where positional-only arguments end:
        
```python
def safe_division(number, divisor, /, *,
                  ignore_overflow=False,
                  ignore_zero_division=False):
    ...
```

One notable consequence of keyword- and positional-only arguments 
is that any parameter name between the / and * symbols 
in the argument list may be passed either by position 
or by keyword 
(which is the default for all function arguments in Python). 

```python
def safe_division_e(numerator, denominator, /,
                    ndigits=10, *,               # Changed
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        fraction = numerator / denominator       # Changed
        return round(fraction, ndigits)          # Changed
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise
            
result = safe_division_e(22, 7)
print(result)

result = safe_division_e(22, 7, 5)
print(result)

result = safe_division_e(22, 7, ndigits=2)
print(result)

>>>
3.1428571429
3.14286
3.14
```

## Item 26: Define Function Decorators with functools.wraps

In [36]:
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper


# This is equivalent to 
# fibonacci = trace(fibonacci)
@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    
    return (fibonacci(n - 2) + fibonacci(n - 1))


fibonacci(4)

fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci((4,), {}) -> 3


3

### Problem

In [39]:
print(fibonacci)

<function trace.<locals>.wrapper at 0x7f5310732f28>


In [40]:
help(fibonacci)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



In [42]:
import pickle

try:
    # Object serializers break because 
    # they can’t determine the location of 
    #the original function that was decorated:
    pickle.dumps(fibonacci)
except Exception as e:
    print(f'{e!r}')

AttributeError("Can't pickle local object 'trace.<locals>.wrapper'")


In [ ]:
# Solution
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
         ...
    return wrapper